In [3]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import re
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import multiprocessing 
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


In [4]:
#¿Las inversiones deben ser reportadas en el anexo es-4?
print('Ingrese la pregunta:')
pregunta = input()

Ingrese la pregunta:
¿Las inversiones deben ser reportadas en el anexo es-4?


In [5]:
cadpun = string.punctuation + '¿�'
stops = stopwords.words('spanish')


pregunta = pregunta.lower()
pregunta = pregunta.translate(str.maketrans('', '', cadpun))
palabras = pregunta.split()
resultwords = [palabra for palabra in palabras if palabra.lower() not in stops]
preguntaclean = (' '.join(resultwords)).split()
preguntaclean

['inversiones', 'deben', 'ser', 'reportadas', 'anexo', 'es4']

In [6]:
tagged_question = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(pregunta)]
palabras = pregunta.split()

In [7]:
model = gensim.models.Word2Vec.load("./doc2vec_wikipedia_es_pvdbow")
model.docvecs.vectors_docs.shape


(1301830, 300)

In [19]:
from pymongo import MongoClient
client = MongoClient('localhost', 27017)

bdtesis = client.BDTesis

documentos = bdtesis.documentosSBS


In [40]:
vector = model.infer_vector(preguntaclean).reshape(1,-1)
vectores = []
documentos = []
capitulos = []
articulos = []
literales1 = []
literales2 = []
documentocod = []
for x in bdtesis.documentosSBS.find():
    vectores.append(x['vector'])
for x in bdtesis.documentosSBS.find():
    documentos.append(x['contenido'])
for x in bdtesis.documentosSBS.find():
    capitulos.append(x['capitulo'])
for x in bdtesis.documentosSBS.find():
    articulos.append(x['articulo'])
    
for x in bdtesis.documentosSBS.find():
    literales1.append(x['literal1'])
    
for x in bdtesis.documentosSBS.find():
    literales2.append(x['literal2'])
    
for x in bdtesis.documentosSBS.find():
    documentocod.append(x['documento'])

In [41]:
#comparación la pregunta ingresada con la BD
proximidades = []
maxprox = 0
cont = 0
similares = {}

for x in vectores:
    vectordoc = np.asarray(x).reshape(1,-1)
    res = cosine_similarity(vectordoc,vector) 
    #similares[float(res[0])] = preguntas[cont]
    similares[documentos[cont]] = float(res[0]) 
    if (res> maxprox):
        docaprox = documentos[cont]
        maxprox = res
        contprox = cont
        capprox = capitulos[cont]
        artprox = articulos[cont]
        lit1prox = literales1[cont]
        lit2prox = literales2[cont]
        docprox = documentocod[cont]
    proximidades.append(res)
    cont = cont + 1
docaprox


'El cálculo del indicador de medición del riesgo cambiario deberá ser incluido en el Anexo ES-19A.'

In [42]:
max(proximidades)

array([[0.58205193]])

In [43]:
ordenados = sorted(similares.items(), key=lambda kv: kv[1], reverse=True)


In [44]:
print(ordenados[0])
print(ordenados[1])
print(ordenados[2])
print(ordenados[3])
print(ordenados[4])


('El cálculo del indicador de medición del riesgo cambiario deberá ser incluido en el Anexo ES-19A.', 0.5820519293093409)
('La unidad de inversiones debe cumplir como mínimo con las siguientes funciones: ', 0.5746024045826336)
('Se aplicarán los siguientes factores:', 0.5665228002660154)
('El Anexo N° ES-4E.2 Características adicionales de las inversiones en inmuebles que generan flujos periódicos se remite con periodicidad semestral ', 0.5648999897652188)
('Anexo ES-19B: Posición en dólares de los Estados Unidos de América.', 0.5544676467992563)


In [47]:
print("Documento: " + str(docprox))
print("Capítulo: " + capprox)
print("Artículo: " + str(artprox))
print("Literal 1: " + str(lit1prox))
print("Literal 2: " + str(lit2prox))

Documento: 2507
Capítulo: II
Artículo: 5
Literal 1: None
Literal 2: None


# Verificación